An Nbody Simulation to test the animation engine

In [ ]:
import sys

sys.path.insert(0, '../')

import physthon as phys
import random as rand
from os import urandom
import numpy as np

# camera always faces the origin, can adjust the fov parameter if needed
renderer = phys.Renderer(900, 600, camera_position=[0, 100, 0])

# seeding the random number generator 
seed = urandom(5)  # for testing purposes can pick a seed to get the same reults each time
# seed = 12345    # can uncomment this line and comment the line above
print("seed = " + str(seed))
rand.seed(seed)

n = 3  # number of objects in scene
G = -110  # m/kg s^2  can be changed, arbitrary central force constant based on mass will change speed of simulation
objects = []

for i in range(n):  # generates n spheres and places them in the scene, each have randome pos and 
    objects.append(
        # random() generates a float from 0 to 1
        phys.Sphere(mass=rand.random() * 100, color='orange', position=[rand.random() * 150 - 75, 0, rand.random() * 150 - 75],
                    velocity=[(rand.random() * 2 - 1) * 10, 0, (rand.random() * 2 - 1) * 10])
    )
    renderer.scene.add(objects[len(objects) - 1].mesh)  # adds the new object mesh to scene

# animation data setup
times = []
animate_pos = []
for i in range(n):  # this will create position lists for all n objects
    animate_pos.append([])

# t = 0 initialization
dt = 0.1
endTime = 20
times.append(0)

for i in range(n): # appends the initial positions of 
    for j in range(3):
        animate_pos[i].append(objects[i].pos[j])

# euler's method 
for t in np.arange(dt, endTime, dt):  # np.arange can have double values while range is only integers
    times.append(t)  # stores time for animation
    for i in range(n):
        objects[i].acc = [0, 0, 0]  # set to 0 as the next for loop will recalculate acceleration
        for j in range(n):  # this whole loop recalculates acceleration of the ith object
            if i != j:
                dx = np.abs(objects[i].pos[0] - objects[j].pos[0])
                dy = np.abs(objects[i].pos[1] - objects[j].pos[1])
                dz = np.abs(objects[i].pos[2] - objects[j].pos[2])
                r = np.sqrt(dx ** 2 + dy ** 2 + dz ** 2)
                f_net = G * objects[i].mass * objects[j].mass / r ** 2
                f = phys.vector_components(f_net, dx, dy, dz)
                for s in range(3):
                    objects[i].acc[s] += f[s] / objects[i].mass
        objects[i].pos = phys.euler_method(dt, objects[i].pos, objects[i].vel, objects[i].acc)
        for s in range(3):  # appends current position to objects animation list
            animate_pos[i].append(objects[i].pos[s])

animators = []
for i in range(n):
    animators.append(phys.PositionAnimator(objects[i].mesh, times, animate_pos[i]))

renderer.render()


In [ ]:
for i in range(n):
    animators[i].animate()